In [29]:
import os, sys, json
from xml.etree.ElementTree import parse
from urllib.parse import urlencode, unquote, quote_plus
import xmltodict
import pandas as pd

if sys.version_info[0]==3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


BASE_DIR = './'

# Secret File Load
secret_file = os.path.join(BASE_DIR,'secret.json')
with open(secret_file) as f:
    secrets = json.loads(f.read())

KEY = unquote(secrets['Enecoding'])

**setting**
bulidingtype = 0 : total, 1: apt, 3 : ali, 7 : sgl
regio = city(36000 : sejong, A1000 : 전국, A2000 : 수도권, 11000 : seoul)
startmonth/endmonth = yyyymm

In [319]:
from itertools import product

b_type = ['0','1','3','7']
b_region = ['36000', 'A1000', 'A2000', '11000']
list_com = list(product(b_region,b_type))

In [320]:
def call_open_api(region,_type) : 

    URL = 'http://openapi.reb.or.kr/OpenAPI_ToolInstallPackage/service/rest/TradePrcIndexSvc/getTradePrcIndex'
    queryParams = '?' + urlencode(
        {
            quote_plus('serviceKey') : KEY,
            quote_plus('startmonth') : '201701',
            quote_plus('endmonth') : '202102',
            quote_plus('region') : region,
            quote_plus('buildingtype') : _type,

        }
    )

    response = urlopen(URL + queryParams).read()
    decode_data = response.decode('utf-8')
    
    xml_parse =  xmltodict.parse(decode_data)
    xml_dict = json.loads(json.dumps(xml_parse))

    return xml_dict

# cc = xmltodict.parse(tree) # return collections.OrderedDict
# dd = json.loads(json.dumps(cc)) # return dict
# animals = dd['animals']['animal']
# print(animals) # 결과를 출력한다

In [340]:
def writeDatafram(region, _type) :

    new_data = call_open_api(region, _type)
    new_data = new_data['response']['body']['item']
   
    df = pd.Series(new_data)
    
    return df

In [343]:
sample = pd.DataFrame()
for i in list_com :

    (region, _type) = i
    df = writeDatafram(region, _type)

    sample = sample.append(df, ignore_index=True)

sample



,aptTypeNm,regionCd,regionNm,rsRow
0,전체,36000,세종,"201701,96.0|201702,96.0|201703,96.0|201704,96...."
1,아파트,36000,세종,"201701,95.9|201702,95.8|201703,95.7|201704,95...."
2,연립/다세대,36000,세종,"201701,99.4|201702,99.4|201703,99.3|201704,99...."
3,단독,36000,세종,"201701,96.0|201702,96.2|201703,96.5|201704,96...."
4,전체,A1000,전국,"201701,98.7|201702,98.7|201703,98.7|201704,98...."
5,아파트,A1000,전국,"201701,99.0|201702,99.0|201703,99.0|201704,99...."
6,연립/다세대,A1000,전국,"201701,99.0|201702,99.0|201703,99.1|201704,99...."
7,단독,A1000,전국,"201701,97.6|201702,97.7|201703,97.8|201704,98...."
8,전체,A2000,수도권,"201701,97.9|201702,98.0|201703,98.0|201704,98...."
9,아파트,A2000,수도권,"201701,97.7|201702,97.7|201703,97.8|201704,97...."
